In [ ]:
# @title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License

# Use windowing with RunInference predictions 

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/apache/beam/blob/master/examples/notebooks/beam-ml/run_inference_windowing.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/apache/beam/blob/master/examples/notebooks/beam-ml/run_inference_windowing.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png" />View source on GitHub</a>
  </td>
</table>


This notebook shows how to use the RunInference transform with [windowing](https://beam.apache.org/documentation/programming-guide/#windowing) in a streaming pipeline. Windowing is useful when your data arrives within a particular timeframe and can be divided by timestamp, or when you want to see trends before all the data is processed. In this example, the pipeline predicts the quality of milk samples and classifies them as `good`, `bad`, or `medium`. The pipeline then aggregates the predictions for each window. To make predictions, the pipeline uses the XGBoost model handler. For more information about the RunInference API, see the [Machine Learning section of the Apache Beam documentation](https://beam.apache.org/documentation/ml/overview/).

With RunInference, a model handler manages batching, vectorization, and prediction optimization for your XGBoost pipeline or model.

This notebook demonstrates the following common RunInference patterns:

- Generate predictions for all samples in a window.
- Aggregate the results per window after running inference.
- Print the aggregations.

## Before you begin
Complete the following setup steps:
- Install dependencies for Apache Beam.
- Install XGBoost.
- Download the [Milk Quality Prediction dataset from Kaggle](https://www.kaggle.com/datasets/cpluzshrijayan/milkquality). Name the dataset `milk_quality.csv`, and put it in the current directory. Use the CSV file format for the dataset. If using colab, you will need to [upload it to the colab filesystem](https://neptune.ai/blog/google-colab-dealing-with-files).

In [ ]:
!pip install apache-beam>=2.47.0
!pip install xgboost
# You may need to install a different version of Datatable directly depending on environment
!pip install datatable

## About the dataset

This dataset is a CSV file that contains seven columns: `pH`, `temperature`, `taste`, `odor`, `fat`, `turbidity`, and `color`. The dataset also contains a column that labels the quality of each sample as `good`, `bad`, or `medium`.

In [ ]:
import argparse
import logging
import time
from typing import NamedTuple

import pandas
from sklearn.model_selection import train_test_split

import apache_beam as beam
import xgboost
from apache_beam import window
from apache_beam.ml.inference import RunInference
from apache_beam.ml.inference.base import PredictionResult
from apache_beam.ml.inference.xgboost_inference import XGBoostModelHandlerPandas
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.runners.runner import PipelineResult
from apache_beam.testing.test_stream import TestStream

## Load the dataset and train the XGBoost model
This section demonstrates the following steps:
1. Load the Milk Quality Prediction dataset from Kaggle.
2. Split the data into a training set and a test set.
2. Train the XGBoost classifier to predict the quality of milk.
3. Save the model in a JSON file using `mode.save_model`. For more information, see [Introduction to Model IO](https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html) in the XGBoost documentation. 


In [ ]:
# Replace with the path to milk_quality.csv
DATASET = "milk_quality.csv"

TRAINING_SET = "training_set.csv"
TEST_SET = "test_set.csv"
LABELS = "labels.csv"
MODEL_STATE = "model.json"

### Split the data into training and test sets 
Use the following preprocessing helper functions to split the dataset into a training set and a test set.

In [ ]:
def preprocess_data(
    dataset_path: str,
    training_set_path: str,
    labels_path: str,
    test_set_path: str):
  """
    Helper function to split the dataset into a training set
    and its labels and a test set. The training set and
    its labels are used to train a lightweight model.
    The test set is used to create a test streaming pipeline.
    Args:
        dataset_path: path to the CSV file containing the Kaggle
         Milk Quality Prediction dataset
        training_set_path: path to output the training samples
        labels_path:  path to output the labels for the training set
        test_set_path: path to output the test samples
    """
  df = pandas.read_csv(dataset_path)
  df['Grade'].replace(['low', 'medium', 'high'], [0, 1, 2], inplace=True)
  x = df.drop(columns=['Grade'])
  y = df['Grade']
  x_train, x_test, y_train, _ = \
      train_test_split(x, y, test_size=0.60, random_state=99)
  x_train.to_csv(training_set_path, index=False)
  y_train.to_csv(labels_path, index=False)
  x_test.to_csv(test_set_path, index=False)


def train_model(
    samples_path: str, labels_path: str, model_state_output_path: str):
  """Function to train the XGBoost model.
    Args:
      samples_path: path to the CSV file containing the training data
      labels_path: path to the CSV file containing the labels for the training data
      model_state_output_path: path to store the trained model
  """
  samples = pandas.read_csv(samples_path)
  labels = pandas.read_csv(labels_path)
  xgb = xgboost.XGBClassifier(max_depth=3)
  xgb.fit(samples, labels)
  xgb.save_model(model_state_output_path)
  return xgb

### Preprocess the data and train the model

Split the dataset into three files with data: a training dataset, a test dataset, and labels for the training dataset. Use the test set as input data for the test stream and to validate the trained model's performance. Use the training set to train the XGBoost model. Store the model in a JSON file so that the model handler can load the model.

In [ ]:
preprocess_data(
    dataset_path=DATASET,
    training_set_path=TRAINING_SET,
    labels_path=LABELS,
    test_set_path=TEST_SET)

train_model(
    samples_path=TRAINING_SET,
    labels_path=LABELS,
    model_state_output_path=MODEL_STATE)

In [ ]:
# Named tuple to store the number of good, bad, and medium quality samples in a window
class MilkQualityAggregation(NamedTuple):
  bad_quality_measurements: int
  medium_quality_measurements: int
  high_quality_measurements: int

### Count the samples by quality for each window 
Use the helper `CombineFn` to aggregate the results of a window. The function tracks of the number of good, bad, and medium quality samples in the stream. 

In [ ]:
class AggregateMilkQualityResults(beam.CombineFn):
  """Simple aggregation to keep track of the number
   of samples with good, bad, and medium quality milk."""
  def create_accumulator(self):
    return MilkQualityAggregation(0, 0, 0)

  def add_input(
      self, accumulator: MilkQualityAggregation, element: PredictionResult):
    quality = element.inference[0]
    if quality == 0:
      return MilkQualityAggregation(
          accumulator.bad_quality_measurements + 1,
          accumulator.medium_quality_measurements,
          accumulator.high_quality_measurements)
    elif quality == 1:
      return MilkQualityAggregation(
          accumulator.bad_quality_measurements,
          accumulator.medium_quality_measurements + 1,
          accumulator.high_quality_measurements)
    else:
      return MilkQualityAggregation(
          accumulator.bad_quality_measurements,
          accumulator.medium_quality_measurements,
          accumulator.high_quality_measurements + 1)

  def merge_accumulators(self, accumulators: MilkQualityAggregation):
    return MilkQualityAggregation(
        sum(
            aggregation.bad_quality_measurements
            for aggregation in accumulators),
        sum(
            aggregation.medium_quality_measurements
            for aggregation in accumulators),
        sum(
            aggregation.high_quality_measurements
            for aggregation in accumulators),
    )

  def extract_output(self, accumulator: MilkQualityAggregation):
    return accumulator

### Create a streaming pipeline using the test data

Construct a [`TestStream`](https://beam.apache.org/releases/javadoc/2.1.0/org/apache/beam/sdk/testing/TestStream.html) class that contains all samples from the test set. A test stream is testing input that generates an unbounded [`PCollection`](https://beam.apache.org/releases/javadoc/2.1.0/org/apache/beam/sdk/values/PCollection.html) of elements, advancing the watermark and processing time as elements are emitted.

In [ ]:
milk_quality_data = pandas.read_csv(TEST_SET)

start = time.mktime(time.strptime('2023/06/29 10:00:00', '%Y/%m/%d %H:%M:%S'))

# Create a test stream
test_stream = TestStream()

# Watermark is set to 10:00:00
test_stream.advance_watermark_to(start)

# Split the DataFrame into individual samples
samples = [
    milk_quality_data.iloc[i:i + 1] for i in range(len(milk_quality_data))
]

for watermark_offset, sample in enumerate(samples, 1):
  test_stream.advance_watermark_to(start + watermark_offset)
  test_stream.add_elements([sample])

test_stream.advance_watermark_to_infinity()

### Create the XGBoost model handler and run the pipeline
This section demonstrates first how to create a model handler, and then how to run the pipeline.

In [ ]:
model_handler = XGBoostModelHandlerPandas(
    model_class=xgboost.XGBClassifier, model_state=MODEL_STATE)

pipeline_options = PipelineOptions().from_dictionary({})

with beam.Pipeline() as p:
  _ = (
      p | test_stream
      | 'window' >> beam.WindowInto(window.SlidingWindows(30, 5))
      | "RunInference" >> RunInference(model_handler)
      | 'Count number of elements in window' >> beam.CombineGlobally(
          AggregateMilkQualityResults()).without_defaults()
      | 'Print' >> beam.Map(print))